## Sales and Warehouse Operation Analysis using Linear Programming
A new strategy is being developed in the #GYD# involving 3 warehouses and the sales and distribution of 6 different products. 
This analysis is meant to inform the board and HOD of the sales prospects of these products and the market’s impact on supply 
and demand with a goal to maximize profits considering the varying warehouse operational limits, product prices, and possible 
changes that could occur.

In [1]:
import numpy as np
import pandas as pd
from pulp import *

In [2]:
warehouse = ['A', 'B', 'C']
product = ['IA', 'IB', 'JA', 'JB', 'KA', 'KB']

In [3]:
unit_price = {
         product[0]: {'A': 76, 'B': 75, 'C': 74},
         product[1]: {'A': 78, 'B': 75, 'C': 76},
         product[2]: {'A': 122, 'B': 123, 'C': 120},
         product[3]: {'A': 120, 'B': 124, 'C': 121},
         product[4]: {'A': 731, 'B': 735, 'C': 732},
         product[5]: {'A': 730, 'B': 732, 'C': 734}
        }

In [4]:
unit_cost = {
         product[0]: 65,
         product[1]: 67,
         product[2]: 104,
         product[3]: 106,
         product[4]: 615,
         product[5]: 612
}

In [5]:
unit_holding_cost = {'cost': 
                     {
    warehouse[0]: 4, 
    warehouse[1]: 3, 
    warehouse[2]: 4}
                    }

In [6]:
unit_transport_cost = { 
                     product[0]: 3.33,
                     product[1]: 3,
                     product[2]: 4,
                     product[3]: 5,
                     product[4]: 3,
                     product[5]: 3.6
}

In [7]:
unit_loading_unloading_cost = {product[0]: 1, product[1]: 1, product[2]: 1, product[3]: 1, product[4]: 1, product[5]: 1}

In [8]:
capacity = {'capacity': {warehouse[0]:45000, warehouse[1]:75000, warehouse[2]:48420}}

In [9]:
demand = {
         product[0]: 15000,
         product[1]: 5000,
         product[2]: 28472,
         product[3]: 33218,
         product[4]: 43365,
         product[5]: 43365
}

In [10]:
unit_profit = {}
for i in warehouse:
    for j in product:
        c = unit_price[j][i]-unit_cost[j]-unit_holding_cost['cost'][i]-unit_transport_cost[j]-unit_loading_unloading_cost[j]
        if j in unit_profit:
            m = {i: round(c,2)}
            unit_profit[j].update(m)
        else:
            n = {j: {i: round(c,2)}}
            unit_profit.update(n)

## Using PuLP

In [11]:
# Variable
supply = pulp.LpVariable.dicts('supply', (product, warehouse), lowBound=0, cat= 'Integer')

In [12]:
# Problem Objective
model = pulp.LpProblem('Maximize Profit', pulp.LpMaximize)

In [13]:
# Objective Function
model += pulp.lpSum([[supply[j][i] * unit_profit[j][i]] for j in product for i in warehouse])

In [14]:
# Demand constraint
for j in product:
    model += pulp.lpSum([supply[j][i] for i in warehouse]) == demand[j]

In [15]:
# Capacity Constraint
for i in warehouse:
    model += pulp.lpSum([supply[j][i] for j in product]) <= capacity['capacity'][i]

In [16]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [17]:
pulp.value(model.objective)

10414820.0

In [18]:
for s in model.variables():
    if s.varValue > 0:
        print(s.name, '=', s.varValue)
        
print('Maximum profit =', value(model.objective))

supply_IA_A = 15000.0
supply_IB_A = 5000.0
supply_JA_A = 25000.0
supply_JA_B = 3472.0
supply_JB_B = 33218.0
supply_KA_B = 38310.0
supply_KA_C = 5055.0
supply_KB_C = 43365.0
Maximum profit = 10414820.0


 # Scenario 2
Due to the unexpected peak in demand of some products they run out-of-stock at the warehouses shown in the table below (in store: 1, out-of-stock: 0)

In [19]:
product_status = {
    product[0]: {'A': 1, 'B': 1, 'C': 0},
    product[1]: {'A': 1, 'B': 0, 'C': 1},
    product[2]: {'A': 1, 'B': 1, 'C': 1},
    product[3]: {'A': 1, 'B': 1, 'C': 1},
    product[4]: {'A': 1, 'B': 0, 'C': 1},
    product[5]: {'A': 0, 'B': 1, 'C': 1}
}

In [20]:
# Variable
supply = pulp.LpVariable.dicts('supply', (product, warehouse), lowBound=0, cat= 'Integer')

In [21]:
# Problem Objective
model2 = pulp.LpProblem('Maximize Profit', pulp.LpMaximize)

In [22]:
# Objective Function
model2 += pulp.lpSum([[supply[j][i] * unit_profit[j][i] * product_status[j][i]] for j in product for i in warehouse])

In [23]:
# Demand Constraint
for j in product:
    model2 += pulp.lpSum([supply[j][i] for i in warehouse]) == demand[j]

In [24]:
# Capacity Constraint
for i in warehouse:
    model2 += pulp.lpSum([supply[j][i] for j in product]) <= capacity['capacity'][i]

In [25]:
model2.solve()
pulp.LpStatus[model2.status]

'Optimal'

In [26]:
pulp.value(model2.objective)

10273270.0

In [27]:
for s in model2.variables():
    if s.varValue > 0:
        print(s.name, '=', s.varValue)
        
print('Scenario 2 Maximum profit =', value(model2.objective))

supply_IA_A = 15000.0
supply_IB_A = 5000.0
supply_JA_B = 28472.0
supply_JB_B = 33218.0
supply_KA_A = 25000.0
supply_KA_C = 18365.0
supply_KB_B = 13310.0
supply_KB_C = 30055.0
Scenario 2 Maximum profit = 10273270.0


# Scenario 3
In addition to the above scenario there is a delay in supply of some products which causes an increase in demand for their substitutes.

In [28]:
demand = {
    'I': 20000, # product I -> product IA AND/OR product IB since they are substitute products
    'J': 61690, # product J -> product JA AND/OR product JB since they are substitute products
    'K': 86730  # product K -> product KA AND/OR product KB since they are substitute products
}

In [29]:
# Variable
supply = pulp.LpVariable.dicts('supply', (product, warehouse), lowBound=0, cat= 'Integer')

In [30]:
# Problem Objective
model3 = pulp.LpProblem('Maximize Profit', pulp.LpMaximize)

In [31]:
# Objective Function
model3 += pulp.lpSum([[supply[j][i] * unit_profit[j][i] * product_status[j][i]] for j in product for i in warehouse])

In [32]:
# Demand Constraint
for d in demand:
    model3 += pulp.lpSum([supply[j][i] for j in product if (j[0] == d) for i in warehouse]) == demand[d]

In [33]:
# Capacity Constraint
for i in warehouse:
    model3 += pulp.lpSum([supply[j][i] for j in product]) <= capacity['capacity'][i]

In [34]:
model3.solve()
pulp.LpStatus[model3.status]

'Optimal'

In [35]:
pulp.value(model3.objective)

10485462.0

In [36]:
for s in model3.variables():
    if s.varValue > 0:
        print(s.name, '=', s.varValue)
        
print('Scenario 3 Maximum profit =', value(model3.objective))

supply_IB_A = 20000.0
supply_JA_A = 25000.0
supply_JA_B = 36690.0
supply_KB_B = 38310.0
supply_KB_C = 48420.0
Scenario 3 Maximum profit = 10485462.0


# Scenario 4
Considering the above scenarios, a contract was eventually signed with a particular customer for the specific delivery.

In [37]:
delivery = {
    'I': {'A': 5000, 'B': 7000, 'C': 8000}, # product I -> product IA AND/OR product IB since they are substitute products
    'J': {'A': 10000, 'B': 27000, 'C': 24690}, # product J -> product JA AND/OR product JB since they are substitute products
    'K': {'A': 30000, 'B': 41000, 'C': 15730} # product K -> product KA AND/OR product KB since they are substitute products
}

In [38]:
# Variable
supply = pulp.LpVariable.dicts('supply', (product, warehouse), lowBound=0, cat= 'Integer')

In [39]:
# Problem Objective
model4 = pulp.LpProblem('Maximize Profit', pulp.LpMaximize)

In [40]:
# Objective Function
model4 += pulp.lpSum([[supply[j][i] * unit_profit[j][i] * product_status[j][i]] for j in product for i in warehouse])

In [41]:
# Demand Constraint
for d in demand:
    model4 += pulp.lpSum([supply[j][i] for j in product if (j[0] == d) for i in warehouse]) == demand[d]

In [42]:
# Delivery Constraint
for d in delivery:
    for i in warehouse:
        model4 += pulp.lpSum([supply[j][i]] for j in product if (j[0] == c)) <= delivery[d][i]

In [43]:
model4.solve()
pulp.LpStatus[model4.status]

'Optimal'

In [44]:
pulp.value(model4.objective)

10573772.0

In [45]:
for s in model4.variables():
    if s.varValue > 0:
        print(s.name, '=', s.varValue)
        
print('Scenario 4 Maximum profit =', value(model4.objective))

supply_IB_A = 20000.0
supply_JA_B = 61690.0
supply_KB_C = 86730.0
Scenario 4 Maximum profit = 10573772.0


In [46]:
print('Maximum profit =', value(model.objective))
print('Scenario 2 Maximum profit =', value(model2.objective))
print('Scenario 3 Maximum profit =', value(model3.objective))
print('Scenario 4 Maximum profit =', value(model4.objective))

Maximum profit = 10414820.0
Scenario 2 Maximum profit = 10273270.0
Scenario 3 Maximum profit = 10485462.0
Scenario 4 Maximum profit = 10573772.0
